# Изучение алгоритмов кластеризации на небольшой выборке

Запуск и просмотр результатов кластеризации различных алгоритмов

In [28]:
import json
import numpy as np
from itertools import groupby
from sklearn.cluster import Birch, AffinityPropagation, MiniBatchKMeans, MeanShift, estimate_bandwidth, KMeans, DBSCAN
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import TruncatedSVD

In [2]:
class News:
    def __init__(self, id, date, title, content, url, siteType):
        self.id = id
        self.date = date
        self.title = title
        self.content = content
        self.url = url
        self.siteType = siteType
    
    @classmethod
    def from_json(cls, json_str):
        json_dict = json.loads(json_str)
        return cls(**json_dict)

In [3]:
news = []
with open('/data/kasandra/year/10k.test.normalized.json', encoding="utf8") as f:
    for line in f:
        news.append(News.from_json(line))

In [4]:
words = []
for n in news:
    words.extend(n.content.split())
counts = Counter(words)
one_time = [k for k, v in dict(counts).items() if v == 1]
print("total words: %s" % (len(words) - len(one_time)))

total words: 2717122


In [5]:
stopwords = set(one_time)

In [6]:
def zip_news(n,l):
    return list(map(assign_label_to_news, zip(n, l)))

def assign_label_to_news(tuplezz):
    (nws, lbl) = tuplezz
    nws.label = lbl.item()
    return nws

def filter_words(text):
    words_list = text.split()
    newWords = [x for x in words_list if len(x) > 3]
    return " ".join(newWords)

def print_clusters(cluster_news, clustre_labels):
    newsLabels = zip_news(cluster_news, clustre_labels)
    newsLabels = sorted(newsLabels, key=lambda n: n.label)
    for label, group in groupby(newsLabels, lambda n: n.label):
        groupList = list(group)
        print("Cluster: %s, count news: %s, titles:" % (label, len(groupList)))
        for gr in groupList:
            print("\t" + gr.title)
            
def print_topics(components, feature_names, n_top_words):
    for topic_idx, topic in enumerate(components):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

# Март 2015

In [7]:
mart_news = news[:500]
mart_content = [filter_words(x.content) for x in mart_news]
print("count mart news: %s" % len(mart_content))

count mart news: 500


In [8]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True, tokenizer=lambda text: text.split(" "), stop_words=stopwords) # , ngram_range=(1, 3)

tfidf_matrix = tfidf_vectorizer.fit_transform(mart_content)
print("vocabulary size: %s" % len(tfidf_vectorizer.vocabulary_))

vocabulary size: 15113


# Birch

In [9]:
km = Birch(n_clusters=50).fit(tfidf_matrix)
labels = km.labels_

In [10]:
print_clusters(mart_news, labels)

Cluster: 0, count news: 34, titles:
	Франция отменила налог на роскошь: вернется ли Депардье на Родину?
	Киевляне установили возле российского посольства деревянные кресты
	Украинские военные требуют отставки Порошенко
	Москаль рассказал о попытках воздействовать на силовиков российской попсой
	Пока силовики обстреливают Донецк, Порошенко призывает к миру
	"Яценюк, выходи!": премьер сыграл с шахтерами в прятки
	«Начать жизнь с нуля»
	На въездах в Киев выставят снайперов
	Под Дебальцево убит «генерал Ичкерии»
	"Дело Carlton": борьба за нравственность или большая политика?
	ООН: количество погибших на Украине превысило 5350 человек
	Кадырова признали главным борцом с пьянством в России
	Штурм администрации Порошенко: первая попытка отражена
	В Киеве митингующие штурмуют здание администрации Порошенко
	Рамзан Кадыров - человек года в борьбе с наркотиками и алкоголизмом в России
	Украина не может счастливой стать - вопреки законам Природы
	США признали отсутствие полного контроля России на

# AffinityPropagation

In [11]:
km = AffinityPropagation().fit(tfidf_matrix)
labels = km.labels_

In [12]:
print_clusters(mart_news, labels)

Cluster: 0, count news: 5, titles:
	Гроза разбудила москвичей утром 1 февраля
	Погода преподнесла сюрприз жителям столичного региона
	Снежную бурю прочувствует треть населения США
	А снег идет: синоптики объявили оранжевый уровень опасности
	Дыхание Атлантики: Россию ожидают снежные выходные
Cluster: 1, count news: 4, titles:
	Кремль поспорил с ополченцами о статусе Кучмы в минских переговорах
	ЛНР и ДНР потребовали пересмотра линии разграничения в Донбассе
	Полпред ЛНР: Контактная группа по Украине взяла тайм-аут
	Главы ДНР и ЛНР: мы готовы остановиться там, где находимся сейчас
Cluster: 2, count news: 3, titles:
	«Резкое повышение невозможно»
	Пять истин, которые помогут вам выжить в кризис
	Мэр Киева объяснил, что подорожавший проезд в метро на самом деле подешевел
Cluster: 3, count news: 4, titles:
	Спасенные от огня книги библиотеки ИНИОН теперь спасают от воды
	ИНИОН — что мы потеряли?
	Пожар в ИНИОНе: специалисты войдут внутрь и оценят масштабы ущерба
	Залитые водой при тушении 

# KMeans

In [13]:
km = KMeans(n_clusters=50, n_jobs=-1).fit(tfidf_matrix)
labels = km.labels_

In [14]:
print_clusters(mart_news, labels)

Cluster: 0, count news: 8, titles:
	Путин потребовал восстановить все отмененные электрички
	Путин потребовал немедленно восстановить отмененные электрички в регионах
	РЖД пообещала Путину как можно скорее вернуть электрички
	Владимир Путин: Перестали ходить электрички! Вы с ума сошли, что ли?
	Путин вернул сбежавшие электрички
	Владимир Путин потребовал восстановить электрички, отменённые в ряде регионов страны
	РЖД восстановили движение 40 отмененных электричек
	РЖД вернет 312 отмененных электричек на следующей неделе
Cluster: 1, count news: 29, titles:
	Кремль поспорил с ополченцами о статусе Кучмы в минских переговорах
	ЛНР и ДНР потребовали пересмотра линии разграничения в Донбассе
	Полпред ЛНР: Контактная группа по Украине взяла тайм-аут
	Пушков: от ухода России из ПАСЕ проиграла Украина
	В Египте приговорили к казни 183 "братьев-мусульман"
	Главы ДНР и ЛНР: мы готовы остановиться там, где находимся сейчас
	"Дело Carlton": борьба за нравственность или большая политика?
	Закон Бож

# MiniBatchKMeans

In [15]:
km = MiniBatchKMeans(n_clusters=50, max_iter=300).fit(tfidf_matrix)
labels = km.labels_

In [16]:
print_clusters(mart_news, labels)

Cluster: 0, count news: 4, titles:
	Россия запретила закупки импортной машиностроительной техники для госнужд
	Семь способов "вырастить" грудь
	Даешь отечественное: товаров из дальнего зарубежья стало на 40% меньше
	Гримасы кризиса: Белорусский сыр по дороге в Россию подорожал в 2,5 раза
Cluster: 1, count news: 8, titles:
	Президент Польши извинился за высказывания Схетыны о Дне Победы
	В Милоша Земана метнули помидор
	Алишер Усманов поможет оплатить долг по зарплате Фабио Капелло
	Усманов дал кредит в 400 млн руб. для погашения долга перед Капелло
	Путин заявил о нежелании России терпеть однополярный мир во главе с США
	Президент Чехии призвал как можно скорее отменить антироссийские санкции
	Президент Чехии высказался за отмену антироссийских санкций
	Социологи зафиксировали в России максимум антизападных настроений
Cluster: 2, count news: 2, titles:
	«Начать жизнь с нуля»
	Украинские солдаты лучше, чем грузинские
Cluster: 3, count news: 5, titles:
	Боевики "Исламского государства" о

# DBScan

In [17]:
db = DBSCAN(eps=1.2, min_samples=3).fit(tfidf_matrix)
labels = db.labels_

print('count clusters: %d' % (len(set(db.labels_)) - (1 if -1 in db.labels_ else 0)))

labels = db.labels_
print("-1: %s, 0: %s" % (labels.tolist().count(-1), labels.tolist().count(0)))

count clusters: 24
-1: 246, 0: 12


In [18]:
print_clusters(mart_news, labels)

Cluster: -1, count news: 246, titles:
	Умерла Джеральдин Макьюэн, сыгравшая мисс Марпл
	Гроза разбудила москвичей утром 1 февраля
	Последний день
	Ким Чен Ын: КНДР готова к ядерной войне
	Франция отменила налог на роскошь: вернется ли Депардье на Родину?
	На алкогольные напитки установили новые минимальные цены
	C сегодняшнего дня в России увеличиваются страховые пенсии
	В центре Киева БТР протаранил два легковых автомобиля
	Погода преподнесла сюрприз жителям столичного региона
	Командира батальона "Донбасс" контузило
	«Резкое повышение невозможно»
	Бойцы "Азова" потеряли четыре танка
	Новосибирский спортсмен с протезом ноги покорил алтайскую гору и стал Альпинистом России
	Игровые трейлеры недели: загробный туризм и ярость ниндзя
	Киевляне установили возле российского посольства деревянные кресты
	Украинские военные требуют отставки Порошенко
	Скидок больше не будет
	Патриарх Кирилл: Церковь не должна жить в своем гетто
	На этапе Кубка мира Вик Уайлд занял первое место в параллельном 

# MeanShift

In [19]:
X = tfidf_matrix.toarray()
bandwidth = estimate_bandwidth(X, quantile=0.2)
km = MeanShift(bandwidth=bandwidth, bin_seeding=True).fit(X)
labels = km.labels_

In [20]:
print_clusters(mart_news, labels)

Cluster: 0, count news: 500, titles:
	Боевики "Исламского государства" обезглавили второго японского заложника
	Умерла Джеральдин Макьюэн, сыгравшая мисс Марпл
	Исламисты опубликовали видео с казнью второго японского заложника
	Кремль поспорил с ополченцами о статусе Кучмы в минских переговорах
	Гроза разбудила москвичей утром 1 февраля
	Грузовики МЧС, доставившие продукты и медикаменты на юго-восток Украины, вернулись в Россию
	Последний день
	Ким Чен Ын: КНДР готова к ядерной войне
	ЛНР и ДНР потребовали пересмотра линии разграничения в Донбассе
	Видео убийства Кэндзи Гото: боевики ИГ пообещали, что устроят для Японии кошмар
	Полпред ЛНР: Контактная группа по Украине взяла тайм-аут
	Франция отменила налог на роскошь: вернется ли Депардье на Родину?
	Ополченцы сообщают, что полностью контролируют Дебальцевский котел
	На алкогольные напитки установили новые минимальные цены
	C сегодняшнего дня в России увеличиваются страховые пенсии
	В центре Киева БТР протаранил два легковых автомобил

# LSA

In [24]:
lsa = TruncatedSVD(n_components=50, n_iter=100)
lsa_matrix = lsa.fit_transform(tfidf_matrix)
lsa_labels = [x.argmax() for x in lsa_matrix]

In [25]:
print_clusters(mart_news, lsa_labels)

Cluster: 0, count news: 222, titles:
	Кремль поспорил с ополченцами о статусе Кучмы в минских переговорах
	Последний день
	Ким Чен Ын: КНДР готова к ядерной войне
	ЛНР и ДНР потребовали пересмотра линии разграничения в Донбассе
	Полпред ЛНР: Контактная группа по Украине взяла тайм-аут
	Франция отменила налог на роскошь: вернется ли Депардье на Родину?
	«Резкое повышение невозможно»
	Новосибирский спортсмен с протезом ноги покорил алтайскую гору и стал Альпинистом России
	Игровые трейлеры недели: загробный туризм и ярость ниндзя
	Киевляне установили возле российского посольства деревянные кресты
	Украинские военные требуют отставки Порошенко
	Скидок больше не будет
	Патриарх Кирилл: Церковь не должна жить в своем гетто
	Запад объявил России рейтинговую войну
	Шувалов: правительство не будет "проламывать" болезненные реформы
	Москаль рассказал о попытках воздействовать на силовиков российской попсой
	Шувалов пригрозил ограничениями поднявшим цены торговым сетям
	Картинки недели: как реаг

# LDA with TF

In [29]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, tokenizer=lambda text: text.split(" "), stop_words=stopwords)
tf = tf_vectorizer.fit_transform(mart_content)

In [31]:
lda = LatentDirichletAllocation(n_topics=50, max_iter=100, learning_method='online',
                                learning_offset=50., )
lda_matrix = lda.fit_transform(tf)

In [32]:
lda_labels = [x.argmax() for x in lda_matrix]
print_clusters(mart_news, lda_labels)

Cluster: 1, count news: 58, titles:
	Умерла Джеральдин Макьюэн, сыгравшая мисс Марпл
	Новосибирский спортсмен с протезом ноги покорил алтайскую гору и стал Альпинистом России
	Игровые трейлеры недели: загробный туризм и ярость ниндзя
	На этапе Кубка мира Вик Уайлд занял первое место в параллельном гигантском слаломе
	Дети из принтера: в Эстонии печатают 3D-фигуры малышей в утробе матери на основе данных УЗИ
	Картинки недели: как реагировать на критику и как реанимировать картину после нокаута
	Кот съел колбас и балыков на 6 тысяч гривен
	Анджелину Джоли признали самой обожаемой женщиной планеты
	Из показаний школьного стрелка: 18 раз выстрелил, а попал только в троих! Два балла мне за меткость
	Участница конкурса красоты сорвала корону с головы победительницы. Видео
	Как мозг заставляет нас делать глупости
	У 58-летнего Игоря Саруханова родилась дочь
	Публикация книги о принце Чарльзе вызвала скандал в Великобритании
	Мила Кунис впервые вышла в свет после рождения дочки
	Стартует прием

# LDA with TF-IDF

In [33]:
lda = LatentDirichletAllocation(n_topics=50, max_iter=100, learning_method='online',
                                learning_offset=50., )
lda_matrix = lda.fit_transform(tfidf_matrix)

In [34]:
lda_labels = [x.argmax() for x in lda_matrix]
print_clusters(mart_news, lda_labels)

Cluster: 18, count news: 500, titles:
	Боевики "Исламского государства" обезглавили второго японского заложника
	Умерла Джеральдин Макьюэн, сыгравшая мисс Марпл
	Исламисты опубликовали видео с казнью второго японского заложника
	Кремль поспорил с ополченцами о статусе Кучмы в минских переговорах
	Гроза разбудила москвичей утром 1 февраля
	Грузовики МЧС, доставившие продукты и медикаменты на юго-восток Украины, вернулись в Россию
	Последний день
	Ким Чен Ын: КНДР готова к ядерной войне
	ЛНР и ДНР потребовали пересмотра линии разграничения в Донбассе
	Видео убийства Кэндзи Гото: боевики ИГ пообещали, что устроят для Японии кошмар
	Полпред ЛНР: Контактная группа по Украине взяла тайм-аут
	Франция отменила налог на роскошь: вернется ли Депардье на Родину?
	Ополченцы сообщают, что полностью контролируют Дебальцевский котел
	На алкогольные напитки установили новые минимальные цены
	C сегодняшнего дня в России увеличиваются страховые пенсии
	В центре Киева БТР протаранил два легковых автомоби